<a href="https://colab.research.google.com/github/salonirohil/Task1-Data-Science-Internship/blob/main/Grocery_Shoping_Recommandation_system_User_Interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio # install the missing gradio package

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import gradio as gr

# Load the datasets (update the path accordingly)
groceries = pd.read_csv("/content/groceries.csv")
purchases = pd.read_csv("/content/purchase_history.csv")

# Function to clean item names
def clean_name(name):
    name = re.sub("[^a-zA-Z0-9 ]", "", name)
    return name

# Apply cleaning function to the item names
groceries["clean_name"] = groceries["itemName"].apply(clean_name)

# Vectorize item names
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
tfidf = vectorizer.fit_transform(groceries["clean_name"])

# Function to search for similar items
def search(item_name):
    item_name = clean_name(item_name)
    query_vec = vectorizer.transform([item_name])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = groceries.iloc[indices].iloc[::-1]
    return results

# Function to find similar items based on item ID
def find_similar_items(item_id):
    similar_users = purchases[(purchases["itemId"] == item_id) & (purchases["purchaseCount"] > 2)]["userId"].unique()
    similar_user_recs = purchases[(purchases["userId"].isin(similar_users)) & (purchases["purchaseCount"] > 2)]["itemId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = purchases[(purchases["itemId"].isin(similar_user_recs.index)) & (purchases["purchaseCount"] > 2)]
    all_user_recs = all_users["itemId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]

    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(groceries, left_index=True, right_on="itemId")[["score", "itemName", "categories"]]

# Function to get purchase history for a user
def get_purchase_history(user_id):
    user_purchases = purchases[purchases["userId"] == int(user_id)]
    if user_purchases.empty:
        return pd.DataFrame()  # Return an empty DataFrame if no purchases are found
    user_purchases = user_purchases.merge(groceries, on="itemId", how="left")
    return user_purchases[["itemId", "itemName", "purchaseCount"]]

# Function to update the purchase record
def update_purchase(user_id, item_id, new_count):
    global purchases
    new_count = int(new_count)
    purchases.loc[(purchases["userId"] == int(user_id)) & (purchases["itemId"] == int(item_id)), "purchaseCount"] = new_count
    purchases.to_csv("/content/purchase_history.csv", index=False)
    return f"Updated purchase count for User ID {user_id}, Item ID {item_id} to {new_count}"

# Function to find items by category
def find_items_by_category(category_name):
    category_items = groceries[groceries["categories"].str.contains(category_name, case=False, na=False)]
    return category_items

# Function to display popular items
def display_popular_items():
    popular_items = purchases["itemId"].value_counts().head(10).index
    item_names = groceries[groceries["itemId"].isin(popular_items)]["itemName"]
    item_counts = purchases["itemId"].value_counts().head(10)

    # Create a bar plot for most popular items
    plt.figure(figsize=(10, 6))
    plt.barh(item_names, item_counts, color='skyblue')
    plt.xlabel('Purchase Count')
    plt.title('Top 10 Most Popular Items')
    plt.gca().invert_yaxis()

    # Save the plot to an image file and return it
    plt.savefig('/content/popular_items_plot.png')
    plt.close()  # Close the figure to prevent display in the notebook

    return '/content/popular_items_plot.png'

# Function to handle payment
def confirm_payment(payment_method, card_number=None, expiry=None, cvv=None, paypal_email=None, paypal_password=None, bank_account=None, routing_number=None, other_details=None):
    confirmation = f"Payment Method: {payment_method}\n"
    if payment_method == "Credit/Debit Card":
        confirmation += f"Card Number: {card_number}, Expiry: {expiry}, CVV: {cvv}\n"
    elif payment_method == "PayPal":
        confirmation += f"PayPal Email: {paypal_email}\n"
    elif payment_method == "Bank Transfer":
        confirmation += f"Bank Account: {bank_account}, Routing Number: {routing_number}\n"
    else:
        confirmation += f"Other Details: {other_details}\n"
    confirmation += "Payment Successful! Transaction ID: XYZ123456789"
    return confirmation

# Gradio Interface Components

def search_items(item_name):
    results = search(item_name)
    if not results.empty:
        item_id = results.iloc[0]["itemId"]
        similar_items = find_similar_items(item_id)
        return similar_items
    else:
        return "No similar items found."

def search_items_by_category(category_name):
    results = find_items_by_category(category_name)
    if not results.empty:
        return results[["itemName", "categories"]]
    else:
        return "No items found in this category."

def display_popular_items_interface():
    return display_popular_items()

def get_user_purchase_history(user_id):
    history = get_purchase_history(user_id)
    if not history.empty:
        return history
    else:
        return "No purchase history found for this user."

def update_user_purchase(user_id, item_id, new_count):
    return update_purchase(user_id, item_id, new_count)

# Gradio UI
item_search = gr.Interface(fn=search_items, inputs="text", outputs="dataframe", title="Search Grocery Items")
category_search = gr.Interface(fn=search_items_by_category, inputs="text", outputs="dataframe", title="Find Items by Category")
popular_items = gr.Interface(fn=display_popular_items_interface, inputs=None, outputs=gr.Image(type="filepath"), title="Most Popular Items")
user_history = gr.Interface(fn=get_user_purchase_history, inputs="text", outputs="dataframe", title="User Purchase History")

# Payment interface
payment_interface = gr.Interface(fn=confirm_payment,
                                 inputs=[gr.Dropdown(["Credit/Debit Card", "PayPal", "Bank Transfer", "Other"]),
                                         gr.Textbox(label="Card Number (for card payments)", placeholder="Card Number"),
                                         gr.Textbox(label="Expiry Date (for card payments)", placeholder="MM/YY"),
                                         gr.Textbox(label="CVV (for card payments)", placeholder="CVV"),
                                         gr.Textbox(label="PayPal Email (for PayPal)", placeholder="PayPal Email"),
                                         gr.Textbox(label="PayPal Password (for PayPal)", placeholder="Password", type="password"),
                                         gr.Textbox(label="Bank Account Number (for bank transfer)", placeholder="Bank Account Number"),
                                         gr.Textbox(label="Routing Number (for bank transfer)", placeholder="Routing Number"),
                                         gr.Textbox(label="Other Payment Details", placeholder="Details for other payment methods")],
                                 outputs="text", title="Payment Confirmation")

# Update purchase interface
update_purchase_interface = gr.Interface(
    fn=update_user_purchase,
    inputs=[
        gr.Textbox(label="User ID", placeholder="Enter User ID"),
        gr.Textbox(label="Item ID", placeholder="Enter Item ID"),
        gr.Textbox(label="New Count", placeholder="Enter New Purchase Count")
    ],
    outputs="text",
    title="Update Purchase Record"
)

# Launch Gradio App with Tabs in Sequence Flow
app = gr.TabbedInterface(
    [item_search, category_search, popular_items, user_history, payment_interface, update_purchase_interface],
    ["Search Items", "Category Search", "Popular Items", "User History", "Payment", "Update Purchase"]
)
app.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://11b6f08e451b2ea45d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
